# **Traffic Incidents optimization**

## **Context**

## **Objectives**

## **Model formulation**

## **Model implementation**

Imports

In [2]:
import cvxpy as cp
import numpy as np
# np.set_printoptions(precision=3, suppress=True)

First step is to maximize the severities of the incidents under current response resources(Number of available agents)

In [ ]:
i = 3 # number of agents
j = 5  # number of incidents

b = np.array([1, 2, 3, 2, 1]) # agents needed per incidents

In [7]:
print("Agents needed:\n", b, end="\n")

Agents needed:
 [1 2 3 2 1]


In [8]:
x = cp.Variable((i, j), boolean=True) # agents-incidents matrix
z = cp.Variable(j, boolean=True) # incidents coverage vector

objective = cp.Maximize(b@z) 

constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve(verbose=1)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 22 03:44:06 PM: Your problem has 20 variables, 2 constraints, and 0 parameters.
(CVXPY) Jun 22 03:44:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 22 03:44:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 22 03:44:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 22 03:44:06 PM: Compiling problem (target solver=SCIP).
(CVXPY) Jun 22 03:44:06 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Cone

3.0

In [10]:
print("Agents :\n", x.value, end="\n\n")
print("Incidents:\n", z.value)

Agents :
 [[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

Incidents:
 [0. 0. 1. 0. 0.]


Next step is to minize the total travel time of all the agents

In [13]:
t = np.random.random((i, j))  # travel times matrix
t

array([[0.442, 0.677, 0.253, 0.004, 0.185],
       [0.465, 0.379, 0.927, 0.277, 0.842],
       [0.551, 0.216, 0.514, 0.136, 0.799]])

In [14]:
objective = cp.Minimize(cp.sum(cp.sum(cp.multiply(x, t), axis=0)))


constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z.value), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve()

1.693996824207376

In [15]:
x.value

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

## **Model in practice**

In [1]:
import pandas as pd

In [2]:
agents = pd.read_csv("agents_may.csv")
incidents = pd.read_csv("incidents_may.csv")

In [3]:
incidents.head()

,id,source,type,state,priority,gravity,class,object,latitude,longitude,...,address,zone_id,orient,incident_time,incident_day,incident_weekday,incident_hour,implicated_type,injured_count,dead_count
0,57518,NUSE,Choque,Cerrado,2,Simple,Siniestro,NaN,4.705279,-74.127686,...,Carrera 110 Calle 69 B,Zona 1,Sur - Norte,2022-04-30 23:59:00,30,5,23,Automóvil,0.0,0.0
1,57518,NUSE,Choque,Cerrado,2,Simple,Siniestro,NaN,4.705279,-74.127686,...,Carrera 110 Calle 69 B,Zona 1,Sur - Norte,2022-04-30 23:59:00,30,5,23,Automóvil,0.0,0.0
2,57519,NUSE,Choque,Cerrado,1,Simple,Siniestro,NaN,4.600571,-74.143379,...,Carrera 68I Calle 43 B SUR,Zona 2,Norte - sur,2022-04-30 23:54:00,30,5,23,Automóvil,0.0,0.0
3,57519,NUSE,Choque,Cerrado,1,Simple,Siniestro,NaN,4.600571,-74.143379,...,Carrera 68I Calle 43 B SUR,Zona 2,Norte - sur,2022-04-30 23:54:00,30,5,23,Automóvil,0.0,0.0
4,57519,NUSE,Choque,Cerrado,1,Simple,Siniestro,NaN,4.600571,-74.143379,...,Carrera 68I Calle 43 B SUR,Zona 2,Norte - sur,2022-04-30 23:54:00,30,5,23,Automóvil,0.0,0.0


In [4]:
agents.head()

,id,dev_id,read_time,time_stamp,speed,grp_name,localidad,latitude,longitude,devicegroupid
0,1651381596,868033050089715,2022-05-01 00:06:36,2022-05-01 00:01:40,0.053087,S60,RAFAEL URIBE URIBE,4.551646,-74.108515,6
1,1651381596,868033050102534,2022-05-01 00:06:36,2022-05-01 00:01:42,0.000000,S60,KENNEDY,4.650030,-74.147322,6
2,1651381596,868033050090085,2022-05-01 00:06:36,2022-05-01 00:01:47,0.000000,S60,RAFAEL URIBE URIBE,4.581688,-74.128348,6
3,1651381596,868033050103508,2022-05-01 00:06:36,2022-05-01 00:01:52,0.000000,S60,CIUDAD BOLIVAR,4.551098,-74.147635,6
4,1651381596,868033050101650,2022-05-01 00:06:36,2022-05-01 00:01:55,0.000000,S60,KENNEDY,4.619840,-74.159668,6
